## Project Airline AI Assistant

Bringing up all together what we'eve learned to make an AI Customer Support assitant for an Airline.


In [36]:
# Getting API Keys
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Initilizing OpenAI
from openai import OpenAI
openai = OpenAI(api_key=api_key)

In [37]:
openai

In [38]:
# Setting a system prompt
system_prompt = "Your are a helpful assistant for an Airline called FlightAI. Give a short, courteous answers no more than 1 sentence. Always be accurate. If you don't know the answer, say so."

In [40]:
def chat(message, history):
  system_message = [{"role":"system", "content": system_prompt}]
  user_message = [{"role":"user", "content":message}]
  messages = system_message + history + user_message

  # Call OpenAI API
  response = openai.chat.completions.create(model="gpt-4o-mini",
                                            messages=messages)
  
  # Return the response
  return response.choices[0].message.content

In [39]:
import gradio as gr
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


## Introducting to Tools

Tools are powerful feature provided by the frontier LLMs.
With tools, we can write a function, and have the LLM call that function as part of its reponse.
Sounds almost spooky.. we're giving it the power to run code on our machine? Well, kinda.


In [41]:
# Create a function that we will be using later

ticket_prices = {"london":"$799", 
                 "paris": "$899",
                 "tokyo": "$1400",
                 "berlin": "$499",
                 "philippines": "$699"}


def get_ticket_price(destination_city):
  print(f"Tool get_ticket_price called for {destination_city}")
  city = destination_city.lower()
  return ticket_prices.get(city, "Unknown")

In [42]:
get_ticket_price("Philippines")

Tool get_ticket_price called for Philippines


'$699'

In [43]:
# This is a particular dictionary structure that is required to describe our function and to pass into our LLM.

price_function = {
  "name": "get_ticket_price",
  "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city.'",
  "parameters": {
    "type": "object",
    "properties": {
      "destination_city": {
        "type":"string",
        "description":"The city that the customer whants to travel to."
      },
    },
    "required": ["destination_city"],
    "additionalProperties": False
  }
}

# This will then include in a list of tools
tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use the Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the oportunity to inform us that it wants us to run a tool.

Here's how the new chat function looks


In [54]:
def chat_tool(message, history):
  system_message = [{"role": "system", "content": system_prompt}]
  user_message = [{"role": "user", "content": message}]
  messages = system_message + history + user_message

  print(messages)
  
  # Call OpenAI API with tools
  response = openai.chat.completions.create(
    model = "gpt-4o-mini",
    messages=messages,
    tools=tools
  )

  if response.choices[0].finish_reason=="tool_calls":
    print("tool_calls")
    message_response = response.choices[0].message
    print("message_response: ", message_response)

    # Extracting the tool
    tool_response , city = handle_tool_call(message_response)
    print("tool_response: ", tool_response)

    # Appending the both tool and the message
    messages.append(message_response)
    messages.append(tool_response)
    print("new messages: ", messages)

    # Call OpenAI API again but this time without the tools
    response = openai.chat.completions.create(
      model="gpt-4o-mini",
      messages=messages
    )

  return response.choices[0].message.content

In [55]:
import json

def handle_tool_call(message_response):
  tool_call = message_response.tool_calls[0]
  arguments = json.loads(tool_call.function.arguments)
  city = arguments.get("destination_city")
  price = get_ticket_price(city)

  tool_response = {
    "role": "tool",
    "content": json.dumps({"destination_city": city, "price": price}),
    "tool_call_id": message_response.tool_calls[0].id
  }

  return tool_response, city

In [ ]:
gr.ChatInterface(fn=chat_tool, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


[{'role': 'system', 'content': "Your are a helpful assistant for an Airline called FlightAI. Give a short, courteous answers no more than 1 sentence. Always be accurate. If you don't know the answer, say so."}, {'role': 'user', 'content': 'Hi'}]
[{'role': 'system', 'content': "Your are a helpful assistant for an Airline called FlightAI. Give a short, courteous answers no more than 1 sentence. Always be accurate. If you don't know the answer, say so."}, {'role': 'user', 'metadata': {'title': None}, 'content': 'Hi'}, {'role': 'assistant', 'metadata': {'title': None}, 'content': 'Hello! How can I assist you today?'}, {'role': 'user', 'content': 'what are the available flights?'}]
[{'role': 'system', 'content': "Your are a helpful assistant for an Airline called FlightAI. Give a short, courteous answers no more than 1 sentence. Always be accurate. If you don't know the answer, say so."}, {'role': 'user', 'metadata': {'title': None}, 'content': 'Hi'}, {'role': 'assistant', 'metadata': {'tit